# Complex body TFA Inversion

This notebook performs the inversion using Levenberg-Marquadt's algorithm of total field anomaly (TFA) data on a regular grid of a model with a complex geometry.

In [110]:
import numpy as np
import matplotlib.pyplot as plt
import cPickle as pickle
import os

from fatiando import utils
from fatiando.gravmag import polyprism
from fatiando.mesher import PolygonalPrism
from fatiando.vis import mpl, myv
from matplotlib import colors, ticker, cm
from IPython.display import Image as img
from matplotlib.mlab import normpdf

### Auxiliary functions

In [111]:
import sys
sys.path.insert(0, '../../code')

import mag_polyprism_functions as mfun

# Input

### Importing model parameters

In [112]:
model_dir = 'data/model_real_obs_pts.pickle'
data_dir = 'data/real_obs_pts/data.pickle'
#data_dir = 'eqlayer/direction_inc65_dec-40_h190m/eqlayer_depth190m.pickle'

In [113]:
with open(model_dir) as w:
        model = pickle.load(w)

### Observation points and observed data

In [114]:
with open(data_dir) as w:
        data = pickle.load(w)

In [115]:
# observed data and observation points
dobs = data['tfa_obs']
xp = data['grid'][0]
yp = data['grid'][1]
zp = data['grid'][2]
N = data['grid'][3]

### Parameters of the initial model

In [116]:
M = 15 # number of vertices per prism
L = 10 # number of prisms
P = L*(M+2) + 1 # number of parameters

# magnetization
incs = model['inc']
decs = model['dec']
intensity = model['intensity']

# depth to the top, thickness and radius
z0 = model['z0']
dz = 800.
r = 650.

# total magnetization
props = {'magnetization': utils.ang2vec(
        intensity, incs, decs)}
rin = np.zeros(M) + r
m0 = np.hstack((rin, np.zeros(2)))
m0 = np.resize(m0, P - 1) 
m0 = np.hstack((m0, dz)) # inicial parameters vector
model0 = mfun.param2polyprism(m0, M, L, z0, props) # list of classes of prisms

varz = 100.
varint = 3.
varinc = 15.
vardec = 15.

param_list = mfun.varying_param(z0, varz, intensity,
                                 varint, incs, varinc, decs, vardec)

# main field
inc, dec = [-21.5, -18.7]

# predict data
d0 = polyprism.tf(xp, yp, zp, model0, inc, dec)

In [117]:
param_list[0]

[-300.0, {'magnetization': array([ 7.61848593,  1.20664963, -9.19253332])}]

### Limits

In [118]:
# limits for parameters in meters
rmin = 100.
rmax = 2000.
x0min = -1500.
x0max = 1500.
y0min = -1500.
y0max = 1500.
dzmin = 200.
dzmax = 1000.

mmin, mmax = mfun.build_range_param(M, L, rmin, rmax, x0min, x0max, y0min, y0max, dzmin, dzmax)

### Variation

In [119]:
# variation for derivatives
deltax = 0.01*np.max(x0max)
deltay = 0.01*np.max(y0max)
deltar = 0.01*np.max(rmax)
deltaz = 0.01*np.max(dzmax)

### Outcropping parameters

In [120]:
# outcropping body parameters
m_out = np.zeros(M + 2)
#m_out = model['param_vec'][:M+2]

### Folder to save the results

In [121]:
mypath = 'data/real_obs_pts/tfa_inversion/multiple_inc%d_dec%d_r%d_int%d_z0%d_dz%d_varz%d_varint%d_varinc%d_vardec%d' % (incs, decs, rin[0], np.linalg.norm(props['magnetization']), z0, dz, varz, varint, varinc, vardec)
if not os.path.isdir(mypath):
   os.makedirs(mypath)

In [122]:
# output of inversion
inversion = dict()

### Regularization parameters

In [123]:
#lamb = th*0.01 # Marquadt's parameter
lamb = 1000.00
dlamb = 10.      # step for Marquadt's parameter

a1 = 0.00007  # adjacent radial distances within each prism
a2 = 0.0001   # vertically adjacent radial distances
a3 = 0.     # outcropping cross-section
a4 = 0.     # outcropping origin
a5 = 0.008     # vertically adjacent origins
a6 = 0.0000001   # zero order Tikhonov on adjacent radial distances
a7 = 0.0005     # zero order Tikhonov on thickness of each prism

In [124]:
delta = np.array([deltax, deltay, deltar, deltaz])
alpha = np.array([a1, a2, a3, a4, a5, a6, a7])

In [125]:
itmax = 100
itmax_marq = 10
tol = 1.0e-4     # stop criterion

In [126]:
inversion['x'] = xp
inversion['y'] = yp
inversion['z'] = zp
inversion['observed_data'] = dobs

In [127]:
inversion['inc_dec'] = [incs, decs]
inversion['z0'] = z0
inversion['initial_dz'] = dz
inversion['intial_r'] = r
inversion['initial_estimate'] = model0
inversion['initial_data'] = d0
inversion['limits'] = [rmin, rmax, x0min, x0max, y0min, y0max, dzmin, dzmax]
inversion['regularization'] = alpha
inversion['tol'] = tol
inversion['main_field'] = [-21.5, -18.7]
inversion['var'] = [varz, varint, varinc, vardec]

### Inversion

In [128]:
inversion_results = []
for p in param_list:
    d_fit, m_est, model_est, phi_list, model_list, res_list = mfun.levmarq_tf(
        xp, yp, zp, m0, M, L, delta,
        itmax, itmax_marq, lamb,
        dlamb, tol, mmin, mmax,
        m_out, dobs, inc, dec,
        p[1], alpha, p[0], dz
    )
    inversion_results.append([d_fit, m_est, model_est, phi_list, model_list, dobs - d_fit, res_list])

it:  0   it_marq:  0   lambda: 1e+03   misfit: 8.78336e+04
it:  1   it_marq:  0   lambda: 1e+02   misfit: 7.42846e+04
it:  2   it_marq:  0   lambda: 1e+01   misfit: 3.34101e+04
it:  3   it_marq:  0   lambda: 1e+00   misfit: 1.13862e+04
it:  4   it_marq:  0   lambda: 1e-01   misfit: 2.26894e+03
it:  5   it_marq:  0   lambda: 1e-02   misfit: 3.56664e+04
it:  5   it_marq:  1   lambda: 1e-01   misfit: 3.55923e+04
it:  5   it_marq:  2   lambda: 1e+00   misfit: 3.32143e+04
it:  5   it_marq:  3   lambda: 1e+01   misfit: 2.45056e+04
it:  5   it_marq:  4   lambda: 1e+02   misfit: 7.11591e+03
it:  5   it_marq:  5   lambda: 1e+03   misfit: 2.23498e+03
it:  6   it_marq:  0   lambda: 1e+02   misfit: 7.02757e+03
it:  6   it_marq:  1   lambda: 1e+03   misfit: 2.20158e+03
it:  7   it_marq:  0   lambda: 1e+02   misfit: 6.70776e+03
it:  7   it_marq:  1   lambda: 1e+03   misfit: 2.16971e+03
it:  8   it_marq:  0   lambda: 1e+02   misfit: 6.61333e+03
it:  8   it_marq:  1   lambda: 1e+03   misfit: 2.13933e+

# Results

In [129]:
inversion['results'] = inversion_results

In [130]:
file_name = mypath+'/inversion.pickle'
with open(file_name, 'w') as f:
    pickle.dump(inversion, f)